In [ ]:
import requests
import json
import time
from openai import OpenAI
# Substitua 'your_openai_api_key' pela sua chave de API da OpenAI
client = OpenAI(api_key='open_api_secret_key')

In [ ]:
# Resenha feita pelo Foundation Model da OpenAI - Chat GPT

def criar_resenha(reviews_file):

    #Carrega o arquivo de reviews feitas pelos usuários da Amazon
    with open(reviews_file,'r') as f:
        reviews_data = json.load(f)
        reviews_contents = reviews_data['reviews_content']

    abstracts = []

    MAX_CHAMADAS = 5
    contador = 0

    for review in reviews_contents:        
        if contador >= MAX_CHAMADAS:
            print("Limite de 2.000 chamadas atingido. Encerrando.")
            break
        # Faz a chamada a api do GPT
      
        try:
            # Faz a chamada à API do GPT
            response = client.chat.completions.create(
                model="gpt-4.1",
                response_format={ "type": "json_object" },
                messages=[
                    {
                        "role": "system",
                        "content": "Create a review or synopsis of a title and rating made by an amazon user about a product and return in the following JSON format containing only the review or synopsis."
                    },
                    {
                        "role": "user",
                        "content": f"The title: {review['title']} and the rating: {review['content']}"
                    }
                ],
                temperature=0.7,
                max_tokens=64,
                top_p=1
            )

            abstract_text = response.choices[0].message.content.strip()
            print(f"({contador+1}) {abstract_text}")

            abstracts.append({
                "review": f"The title: {review['title']} and the rating: {review['content']}",
                "abstract": abstract_text.replace('{\n    \"abstract\": \"', "")
            })

            contador += 1

            # Opcional: pausa pequena para evitar rate limit da OpenAI
            time.sleep(0.1)

        except Exception as e:
            print(f"Erro ao processar review {contador+1}: {e}")
            continue

     # Salva os resultados em um arquivo JSON
    with open('reviews_abstracts.json', 'w') as json_file:
        json.dump({"reviews_abstracts": abstracts}, json_file)

# Chame a função com o caminho para seu arquivo de conteúdo das avaliações
criar_resenha('prepared_trn.json')